In [1]:
# Standard python library
import configparser
import sys
import os
import argparse
from collections import namedtuple

# Local resources
from gref4hsi.scripts import georeference
from gref4hsi.scripts import orthorectification
from gref4hsi.scripts import coregistration
from gref4hsi.utils import parsing_utils, specim_parsing_utils
from gref4hsi.utils import visualize
from gref4hsi.utils.config_utils import prepend_data_dir_to_relative_paths
from gref4hsi.utils.config_utils import customize_config

# Third party
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# From seabeepy/notebooks/flight_runner
import datetime as dt
import os
from pathlib import Path

from config import SETTINGS
from pyodm import Node
from subprocess import CalledProcessError

import seabeepy as sb


In [3]:


# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)



In [4]:
# Parent directories containing flight folders to process
base_dirs = [
    r"/home/notebook/shared-seabee-ns9879k/ntnu/2022",
]

# Directory for temporary files
temp_dir = r"/home/notebook/cogs"

In [5]:
# Run info
run_date = dt.datetime.today()
print(f"Processing started: {run_date}")

Processing started: 2024-04-16 16:17:40.667898


In [6]:
# Get all potential mission folders for NodeODM
# (i.e. folders containing a 'config.seabee.yaml' and an 'capture' subdirectory, but NOT a 'processed' directory)
mission_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "capture")
    and not sb.ortho.check_subdir_exists(f.parent, "processed")
]

In [7]:
mission_list

[PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2022/2022-08-31-Remoy/remoy_202208311040_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2022/2022-08-31-Remoy/remoy_202208311435_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2022/2022-08-31-Remoy/remoy_202208310800_ntnu_hyperspectral_74m')]

In [8]:
# Establish the ancillary data paths, copy to local work space and 
from pathlib import Path
import shutil

geoid_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/geoids/no_kv_HREF2018A_NN2000_EUREF89.tif')
config_template_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/configuration_specim.ini')
lab_calibration_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations')

# Local path geoid
geoid_path = os.path.join(temp_dir, "no_kv_HREF2018A_NN2000_EUREF89.tif")
try:
    shutil.copyfile(geoid_path_minio, geoid_path)
except FileExistsError:
    pass

# Local path for configuration file
config_template_path = os.path.join(temp_dir, "config_template_path_specim.ini")
try:
    shutil.copyfile(config_template_path_minio, config_template_path)
except FileExistsError:
    pass

# Local path for lab calibration
lab_calibration_path = os.path.join(temp_dir, "lab-calibration")
try:
    shutil.copytree(lab_calibration_path_minio, lab_calibration_path)
except FileExistsError:
    pass

In [ ]:
import process_hyperspectral

# Process missions in the list
for specim_mission_folder_minio in mission_list:
    mission_name = specim_mission_folder_minio.name
    print(f"\n################\nProcessing: {mission_name}")
    
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)
    
    try:
        shutil.copytree(specim_mission_folder_minio, specim_mission_folder)
    except FileExistsError:
        pass

    # The config file is read
    config_yaml = os.path.join(specim_mission_folder, "config.seabee.yaml")

    # Start the specim processing (if it does not finish
    process_hyperspectral.main(str(config_yaml), 
                        str(specim_mission_folder), 
                        geoid_path, 
                        config_template_path, 
                        lab_calibration_path,
                        fast_mode = True)
    
    # Processed datacubes are now stored under
    cubes_path = os.path.join(temp_dir, 'mission-data/processed/Output/GIS/HSIDatacubes')
    cubes_path_minio = os.path.join(specim_mission_folder_minio, 'processed/cubes')

    # Processed composites are now stored under
    composites_path = os.path.join(temp_dir, 'mission-data/processed/Output/GIS/RGBComposites')
    # And should be moved to 
    composites_path_minio = os.path.join(specim_mission_folder_minio, 'processed/composites')

    # Move datacube, composites and ancillary data to persistant storage on Minio
    sb.storage.copy_folder(src_fold = cubes_path, dst_fold = cubes_path_minio, client=minio_client, containing_folder = False)
    sb.storage.copy_folder(src_fold = composites_path, dst_fold = composites_path_minio, client = minio_client, containing_folder = False)

    # Delete the local data to make room for next mission content
    #shutil.rmtree(specim_mission_folder)
    
    break
    


################
Processing: remoy_202208311040_ntnu_hyperspectral_74m
/home/notebook/cogs/remoy_202208311040_ntnu_hyperspectral_74m/dem/DEM_downsampled.tif
The file 'orthomosaic_cropped.tif' is used as as reference orthomosaic.


/opt/conda/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


In [11]:
## 3. Convert georeferenced data into reflectance above seafloor

shutil.rmtree(specim_mission_folder)

FileNotFoundError: [Errno 2] No such file or directory: '/home/notebook/cogs/remoy_202208311040_ntnu_hyperspectral_74m'

## 4. Publish to GeoNode